<a href="https://colab.research.google.com/github/yooje153/-/blob/main/Lecture10_Basic_analysis_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 한국복지패널데이터

* <https://www.koweps.re.kr/>
* 주관: 한국보건사회연구원
* 목적: 국내 가구의 경제활동을 연구하여 정책 지원에 반영할 목적으로 조사
* 데이터 전처리 필요 -> 데이터 전처리와 시각화를 연습해봅시다
* source: [데이터 시각화 with 파이썬](http://book.naver.com/bookdb/book_detail.naver?bid=18123838),  [데이터 다운](https://www.kaggle.com/datasets/hongsean/korea-income-and-welfare?resource=download)


In [ ]:
!fc-list | grep "Nanum"


In [ ]:
!apt-get update -qq
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 2. Matplotlib에 폰트 설정
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc

# NanumGothic 폰트 경로
font_path = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"
font_name = font_manager.FontProperties(fname=font_path).get_name()
plt.rc('font', family=font_name)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.rc('axes', unicode_minus = False)

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# data path
dpath = "/content/drive/MyDrive/강의/2022-2 AI를 위한 프로그래밍/data/"

## 데이터 읽기

* `Korea Income and Welfare.csv`

variable | detail | code
:--- | :--- | :---
year | 연구를 수행한 연도 | YYYY
region | 7개 권역별 지역구분 | 1=서울, 2=수도권(인천/경기), 3=부산/경남/울산, 4=대구/경북, 5=대전/충남, 6=강원/충북, 7=광주/전남/전북/제주
income | 연소득 | float64(xxx), 단위: 만원
family_member | 가족수 | int64(xx)
gender | 성별 | 1=남, 2=여
year_born | 태어난 연도 | YYYY
education_level | 교육수준| 1=no education(under 7 yrs-old) 2=no education(7 & over 7 yrs-old) 3=elementary 4=middle school 5=high school 6=college 7=university degree 8=MA 9=doctoral degree
marriage | 혼인상태 | 1=비해당(18세미만), 2=배우자있음, 3=사별, 4=이혼, 5=별거, 6=미혼(18세이상, 미혼모포함), 7=기타
religion | 종교 유무 | 1=있음, 2=없음
occupation | 직업 코드 | float64(xxx) (`job_code_translated.xlsx` 참고)
reasonnoneworker | 직업 없는 이유 | 1=no capable 2=in military service 3=studying in school 4=prepare for school 5=preprare to apply job 6=house worker 7=caring kids at home 8=nursing 9=giving-up economic activities 10=no intention to work 11=others


In [ ]:
# import data

df = pd.read_csv(dpath + "Korea Income and Welfare.csv")
print(df.head())
print(df.info())

In [ ]:
df.isnull().sum()

In [ ]:
df1 = df.replace(' ', np.NaN)
print(df1.isnull().sum())
print(df1.info())

In [ ]:
ref = pd.read_excel(dpath + "job_code_translated.xlsx", usecols=[3,4])
ref.head()

## 데이터 전처리

1. gender : 1이면 '남성', 2이면 '여성'으로 변경하여 gender2로 코딩
2. marriage : 2면 '결혼', 4이면 '이혼', 나머지는 '그외'로 변경하여 marriage2로 코딩
3. occupation : `job_code_translated.xlsx'에서 직업이름정보(job_title)를 추출하고 df에 병합하여 job으로 코딩
4. region : 7개 권역별 지역이름을 문자열로 변경하여 region2로 변경
5. age : 현재 연도에서 태어난 연도를 빼서 신규컬럼 생성
6. religion : 1이면 '종교있음', 2이면 '무교'로 religion2로 표기
7. agegroup : 연령(age)이 30세 미만이면 '청년', 30세 이상이면서 60세 미만은 '중년', 60세 이상이면 '노년'으로 신규컬럼 코딩 후 범주형 변수로 변경하기
8. income : 연소득이 음수인 경우 제거

In [ ]:
# 1. gender : 1이면 '남성', 2이면 '여성'으로 gender2로 코딩 (loc메소드 적용)
df1.loc[df.gender == 1, "gender2"] = "남성"
df1.loc[df.gender == 2, "gender2"] = "여성"
df1.head()

In [ ]:
# 1. gender : 1이면 '남성', 2이면 '여성'으로 gender2로 코딩 (List Comprehension 적용: dictionary and for문 이용)
# List Comprehension 기본구조: [표현식 for 변수 in 리스트]
dic = {1:"남성", 2: "여성"}
df1["gender2"] = [dic[i] for i in df1.gender]
df1.head()

In [ ]:
# 2. marriage : 2면 '결혼', 4이면 '이혼', 나머지는 '그외'라는 문자열로 처리하여 marriage2로 저장 (lambda함수 적용해보기)
# lambda함수 기본구조: lambda x : 표현식

def recoding(x):
    if x == 2:
        return "결혼"
    elif x == 4:
        return "이혼"
    else:
        return "그외"

df1["marriage2"] = df1.marriage.apply(lambda x : recoding(x))
df1.tail()

In [ ]:
# 3. occupation : `job_code_translated.xlsx'에서 직업이름정보(job_title)를 추출하고 df1에 병합하여 job으로 코딩 (merge이용)
df1['occupation'] = df1['occupation'].replace(np.NaN, 0).astype(int)
df1 = pd.merge(df1,ref, how = "left", left_on = "occupation", right_on = "job_code")

df1.tail()

In [ ]:
# 4. region : 7개 권역별 지역이름을 문자열로 표기하여 region2에 저장 (함수 적용)
# 1=서울, 2=인천/경기, 3=부산/경남/울산, 4=대구/경북, 5=대전/충남, 6=강원/충북, 7=광주/전남/전북/제주
def recoding(x):
    if x == 1:
        return "서울"
    elif x == 2:
        return "인천/경기"
    elif x == 3:
        return "부산/경남/울산"
    elif x == 4:
        return "대구/경북"
    elif x == 5:
        return "대전/충남"
    elif x == 6:
        return "강원/충북"
    else :
        return "광주/전남/전북/제주"

df1["region2"] = df1.region.apply(recoding)
df1.head()

In [ ]:
# 4. region : 7개 권역별 지역이름을 문자열로 표기하여 region2에 저장 (dictionary 적용)
# 1=서울, 2=인천/경기, 3=부산/경남/울산, 4=대구/경북, 5=대전/충남, 6=강원/충북, 7=광주/전남/전북/제주
dic = {1:"서울", 2:"인천/경기", 3:"부산/경남/울산", 4:"대구/경북", 5:"대전/충남", 6:"강원/충북", 7:"광주/전남/전북/제주"}
df1["region2"] = [dic[i] for i in df1.region]
df1.head()

In [ ]:
# 5. age : 현재 연도에서 태어난 연도를 빼서 신규컬럼 생성
df1["age"] = 2022 - df1.year_born
df1.head()

In [ ]:
# 5. age : 현재 연도에서 태어난 연도를 빼서 신규컬럼 생성 (현재 연도를 함수로 가져와보기)
# 구글에서 'year in python' 검색
from datetime import date

current_year = date.today().year
print(date.today())
print(current_year)


df1["age"] = current_year - df1.year_born
df1.head()

In [ ]:
# 6. religion : 1이면 '종교있음', 2이면 '무교'로 표기하여 religion2로 코딩 (List Comprehension사용, if else 구문 이용)
# List Comprehension 기본구조: [표현식 for 변수 in 리스트]

# 직접 코드를 구현해보세요.
df1['religion2'] = ['종교있음' if r == 1 else '무교' for r in df1['religion']]

In [ ]:
df1.head()

In [ ]:
# 7. agegroup : 연령(age)이 30세 미만이면 '청년', 30세 이상이면서 60세 미만은 '중년', 60세 이상이면 '노년'으로 신규컬럼 코딩 (함수 이용)

# 직접 코드를 구현해보세요.

def recoding(x):
  if x < 30:
    return "청년"
  elif x >= 30 and x < 60:
    return "중년"
  else:
    return "노년"

df1['agegroup']=df1.age.apply(recoding)
df1.head()

In [ ]:
# 7. 범주형 변수로 순서 카테고리 지정하기

# 직접 코드를 구현해보세요.

In [ ]:
df1.head()

In [ ]:
# 8. income 변수가 음수인 경우 해당 데이터 삭제
df1.income.describe()
df1.drop(df1[df1.income < 0].index, inplace = True)

In [ ]:
# 9. 다음 변수만 모아서 한글변수명을 붙이고 df2에 저장
# year = 연도, age=연령, agegroup=연령대, year_born=생년, gender2=성별, marriage2=결혼상태,
# religion2=종교유무, region=지역, occupation=직업, income=연소득

df2 = df1[["year","age", "agegroup", "year_born", "gender2", "marriage2", "religion2", "region2", "occupation", "income"]]
df2.head()

In [ ]:
 # 컬럼명 확인
 df2.columns

In [ ]:
# 위치가 변경될 수 있으니 dictionary 이용
dic = {"year":"연도","age":"연령", "agegroup":"연령대", "year_born":"생년", "gender2":"성별", "marriage2":"결혼상태", "religion2":"종교유무",
       "region2":"지역", "occupation":"직업", "income":"연소득"}
df2 = df2.copy()
df2.rename(columns=dic, inplace=True)
df2.tail()

In [ ]:
# df2를 dpath/output/welfare_clean.csv로 저장
df2.to_csv(dpath + "output/welfare_clean.csv")

In [ ]:
# welfare_clean.csv를 불러와서 잘 저장되었는지 확인해보기
df = pd.read_csv(dpath + "output/welfare_clean.csv")
df.head()

In [ ]:
?pd.DataFrame.to_csv

In [ ]:
# df2를 dpath/output/welfare_clean.csv로 저장 (index 없이 저장하기)
df2.to_csv(dpath + "output/welfare_clean.csv", index=False)

# welfare_clean.csv를 불러와서 잘 저장되었는지 확인해보기
df = pd.read_csv(dpath + "output/welfare_clean.csv")
df.head()

## 데이터 시각화

### 직업과 소득의 분포

In [ ]:
# 연도별 소득이 높은 직업은 무엇인가?
# 2018년에 소득이 높은 직업이 무엇인지 막대그림으로 시각화하시오
top_j_income = df1.groupby(['year','job_title'], sort = True, as_index=False).sum()
top_j_income = top_j_income.sort_values(by=['year','income'],ascending = [True, False])
top_j_income_2018 = top_j_income[top_j_income['year'] == 2018]
top_j_income_2018 = top_j_income_2018[['job_title', 'income']].head(10)
sns.barplot(x="income",y="job_title",data=top_j_income_2018);

In [ ]:
df1.info()

In [ ]:
# 숫자형 열만 선택
numeric_columns = df1.select_dtypes(include=['number'])

# 그룹화와 평균 연산
top_j_income_job = df1.groupby(['year', 'job_title'], sort=True, as_index=False)[numeric_columns.columns].mean()

# 소득 기준으로 정렬
top_j_income_job = top_j_income_job.sort_values(by=['year', 'income'], ascending=[True, False])

# 2018년 데이터 필터링 및 상위 10개 직업 추출
top_j_income_job_2018 = top_j_income_job[top_j_income_job['year'] == 2018]
top_j_income_job_2018 = top_j_income_job_2018[['job_title', 'income']].head(10)

# 시각화
sns.barplot(x="income", y="job_title", data=top_j_income_job_2018)

In [ ]:
# 높은 임금을 받는 상위 10개 직업의 성별 확인
top_j_income_job = df1.groupby(['year','job_title'], sort = True, as_index=False)[numeric_columns.columns].mean()
top_j_income_job = top_j_income_job.sort_values(by=['year','income'],ascending = [True, False])


top_j_income_job_2018 = top_j_income_job[top_j_income_job['year'] == 2018].head(10)
top_10_job_list = top_j_income_job_2018['job_title'].to_list()
top_10_job_list

In [ ]:
df1_2018= df1[df1['year'] == 2018]
df1_2018_job = df1_2018[df1_2018['job_title'].isin(top_10_job_list)]
df1_2018_job = df1_2018_job.groupby(['job_title','gender'], as_index=False).sum()
df1_2018_job


In [ ]:
sns.barplot(x="income",y="job_title",data=df1_2018_job, hue='gender');

In [ ]:
# 데이터의 전체 평균 소득보다 소득이 낮은 직업은 무엇인가요?
total_mean = df1[df1['occupation'] != 0].income.mean()
print(total_mean)

job_mean = df1.groupby('job_title')['income'].mean()
job_over_mean = job_mean[job_mean <= total_mean].sort_values(ascending=True)
print(job_over_mean)

In [ ]:
n = 10
job_list = job_over_mean.head(n).index.tolist()
job_list.append('mean')
x_pos = np.arange(n+1)
income_list = job_over_mean.head(n).tolist()
income_list.append(total_mean)

bar_list = plt.bar(x_pos, income_list, align='center', alpha=0.5)
bar_list[n].set_color('r')
plt.plot([0.,n], [total_mean, total_mean], 'k--')
plt.xticks(x_pos, labels=job_list, rotation='vertical')

plt.ylabel('Income')
plt.title('Job income list under mean')

plt.show()

### 결혼상태와 종교유무의 분포

In [ ]:
# 결혼상태별 빈도 확인
df["결혼상태"].value_counts()

In [ ]:
# 빈도막대그래프 그리기
sns.catplot(kind="count", x="결혼상태", data=df);

In [ ]:
# 결혼상태 순서를 정해주기
df["결혼상태"] = pd.Categorical(df["결혼상태"], categories=["결혼", "이혼", "그외"])

In [ ]:
sns.catplot(kind="count", x="결혼상태", data=df);

In [ ]:
# countplot()의 order 옵션 사용해서 이혼, 결혼, 그외 순으로 그림 그려보기
sns.catplot(kind="count", x="결혼상태", data=df, order=["이혼", "결혼", "그외"]);

In [ ]:
# 결혼상태에 따라 종교유무를 나누어 표기
# 막대 색상을 팔레트 "Set2" 이용
sns.countplot(x="결혼상태", data=df, hue="종교유무", palette="Set2");

In [ ]:
# 결혼상태에 따라 종교유무를 나누어 표기 (종교있음 > 무교 순으로 표기)
# 테두리 두께를 1로 지정하고, 테두리 색상을 navy로 주기
sns.countplot(x="결혼상태", data=df, hue="종교유무", hue_order=["종교있음", "무교"],
              linewidth=1, edgecolor="navy");

In [ ]:
# 위 그림을 가로막대로 그리고, 막대 색상을 팔레트 "Paired"로 주고, 테두리 색상을 gray로 주기
sns.countplot(y="결혼상태", data=df, hue="종교유무", hue_order=["종교있음", "무교"], palette="Paired", edgecolor="gray");

### 연령 분포


In [ ]:
# 연령에 대한 히스토그램
sns.displot(kind="hist", x="연령", data=df, binwidth=5);

In [ ]:
# 연령에 대한 히스토그램에 밀도곡선 얹고, 색상을 magenta로 지정하기
sns.displot(kind="hist", x="연령", data=df, binwidth=5, kde=True, color="m");

### 연령과 소득, 결혼유무 분포

In [ ]:
# 연령과 연소득의 산점도
sns.relplot(kind="scatter", x="연령", y="연소득", data=df);

In [ ]:
# 연령과 월소득의 산점도 그린 뒤, 마진에 히스토그램 얹기
sns.jointplot(x="연령", y="연소득", data=df);

In [ ]:
# pairplot으로 그림 그리기
sns.pairplot(data=df[["연령", "연소득"]]);

In [ ]:
# pairplot의 대각선은 밀도곡선으로 변경하고, 하삼각만 그리기
sns.pairplot(data=df[["연령", "연소득"]], diag_kind="kde", corner=True);

In [ ]:
# 연령과 월소득에 대한 pairplot을 결혼상태에 따라 색상을 구분하여 그리기
sns.pairplot(data=df[["연령", "연소득", "결혼상태"]], hue="결혼상태");

In [ ]:
# 연령과 월소득에 대한 산점도를 결혼상태에 따라 나눠서 그리되, 결혼상태는 결혼과 이혼만 추출해서 그리기
sns.relplot(kind="scatter", x="연령", y="연소득", data=df.query("결혼상태 == '결혼' | 결혼상태 == '이혼'"),
            col="결혼상태", col_order=["결혼", "이혼"]);

### 성별과 연령 분포

In [ ]:
# 연령에 대한 박스그림
sns.catplot(kind="box", x="연령", data=df, width=0.5);

In [ ]:
# 성별에 따라 연령에 대한 바이올린 그림 그리기
sns.violinplot(x="성별", y="연령", data=df);

In [ ]:
# 성별에 따른 연령분포의 바이올린 그림을 결혼상태에 따라 나눠 그리기 (단, 결혼상태는 '결혼', '그외'만 추출하여 그리기)
sns.violinplot(x="성별", y="연령", data=df.query("결혼상태 != '이혼'"), hue="결혼상태", hue_order=["결혼", "그외"]);

In [ ]:
# 성별에 따른 연령분포의 바이올린 그림을 결혼상태에 따라 분할된(반반) 바이올린으로 그리기 (단, 결혼상태는 '결혼', '그외'만 추출하여 그리기)
sns.violinplot(x="성별", y="연령", data=df.query("결혼상태 != '이혼'"), hue="결혼상태", hue_order=["결혼", "그외"], split=True);

### 성별과 소득 분포

In [ ]:
# 성별에 따른 평균 월소득 계산
df.groupby("성별")["연소득"].mean()

In [ ]:
df["연소득"].groupby(df["성별"]).mean()

In [ ]:
# 성별에 따른 평균 연소득에 대한 평균막대그래프 (신뢰구간 표시 안하기)
sns.catplot(kind="bar", x="성별", y="연소득", data=df, ci=None);

In [ ]:
# 성별에 따른 평균 연소득에 대한 평균막대그래프를 결혼상태에 따라 구분하여 가로막대로 그리기
sns.catplot(kind="bar", y="성별", x="연소득", data=df, ci=None, hue="결혼상태");

In [ ]:
# 성별에 따른 연소득에 대한 박스그림 그리기
sns.catplot(kind="box", x="성별", y="연소득", data=df);

In [ ]:
# 성별에 따른 연소득에 대한 박스그림 그리기 (평균 연소득이 4000 이하인 개체만 추출)
sns.catplot(kind="box", x="성별", y="연소득", data=df.query("연소득<=4000"));

# 서울시 구별 CCTV 현황 분석

* [서울시 자치구 연도별 CCTV 설치 현황 데이터](http://data.seoul.go.kr/dataList/OA-2734/F/1/datasetView.do)
* [서울시 자치구 연도별 인구 데이터](https://data.seoul.go.kr/dataList/10790/S/2/datasetView.do)
* 서울시 구별 CCTV 설치현황(구별 인구대비현황 등) 분석
* source: [파이썬으로 데이터 주무르기](https://github.com/PinkWink/DataScience)



## 데이터 읽기

In [ ]:
# df = pd.read_csv(dpath + "서울시CCTV설치운영현황(자치구)_년도별_210731기준.csv", encoding="CP949", skiprows=1)

# thousands = ',' 옵션으로 천 단위 자리수 구분 콤마 없애고 불러오기
df = pd.read_csv(dpath + "서울시CCTV설치운영현황(자치구)_년도별_210731기준.csv", encoding="CP949", skiprows=1, thousands=",")
df.head()

In [ ]:
print(df.info())
print("-----")
print(df.shape)

In [ ]:
# 구분이 "계"인 행 삭제
df = df[df["구분"] != "계"]
df.head()

In [ ]:
# 2021년 열 삭제
df.drop("2021년", axis=1, inplace=True)
df.head()

In [ ]:
pop = pd.read_excel(dpath + "population_in_Seoul.xlsx", skiprows=2,usecols=["기간", "자치구", "계", "계.1", "계.2", "65세이상고령자"])
pop.head()

In [ ]:
pop.rename(columns={"계":"인구수", "계.1":"한국인", "계.2":"외국인", "65세이상고령자":"고령자"}, inplace=True)
pop.head()

In [ ]:
# 자치구가 "합계"인 행 삭제
pop = pop[pop["자치구"] != "합계"]
pop.head()

In [ ]:
# 자치구별 데이터 개수 확인
pop.groupby("자치구")["기간"].count()

## 데이터 파악



### CCTV 현황

In [ ]:
# CCTV 데이터에서 CCTV 전체개수로 정렬해서 CCTV가 가장 적은 구부터 5개구 확인하기
df.head()
df.sort_values("총계").head()

In [ ]:
# CCTV 데이터에서 CCTV 전체개수로 정렬해서 CCTV가 가장 많은 구부터 5개구 확인하기
df.head()
df.sort_values("총계", ascending=False).head()

In [ ]:
# 최근 3년간 CCTV 증가율 계산 (최근 3년간 CCTV 수를 더하고, 최근 3년 이전 CCTV 수로 나누기)

df["최근3년CCTV수"] = df[["2018년", "2019년", "2020년"]].sum(axis=1)  # 행별로 컬럼의 합 구하기
df["최근3년이전CCTV수"] = df.loc[:, "2012년 이전":"2017년"].sum(axis=1)
df["최근3년CCTV증가율"] = df["최근3년CCTV수"] / df["최근3년이전CCTV수"] * 100
df.head()

In [ ]:
df.drop(["최근3년CCTV수", "최근3년이전CCTV수"], axis=1, inplace=True)
df.head()

In [ ]:
# 최근 3년 CCTV 증가율이 가장 높았던 구부터 5개구 확인
df.sort_values("최근3년CCTV증가율", ascending=False).head()

### 서울시 인구현황

In [ ]:
pop.head()

In [ ]:
# 구별 외국인비율과 고령자비율 계산

pop["외국인비율"] = pop["외국인"] / pop["인구수"] * 100
pop["고령자비율"] = pop["고령자"] / pop["인구수"] * 100
pop.head()

In [ ]:
# 인구수가 가장 많은 구부터 5개구 확인
pop.sort_values("인구수", ascending=False).head()

In [ ]:
# 외국인 비율이 가장 높은 구부터 5개구 확인
pop.sort_values("외국인비율", ascending=False).head()

In [ ]:
# 고령자 비율이 가장 높은 구부터 5개구 확인
pop.sort_values("고령자비율", ascending=False).head()

## 데이터 병합

In [ ]:
# CCTV 데이터에서 불필요한 부분 삭제
df.head()

In [ ]:
x = df.loc[:, "2012년 이전":"2020년"].columns

In [ ]:
df.drop(x, axis=1, inplace=True)
df.head()

In [ ]:
# 서울시 데이터에서 불필요한 부분 삭제 (자치구, 인구수, 한국인, 외국인, 고령자, 외국인비율, 고령자비율만 남기기)
pop.head()

In [ ]:
pop = pop.loc[:, "자치구":]
pop.head()

In [ ]:
# 두 데이터 병합 (구 이름 기준)
final = pd.merge(df, pop, left_on="구분", right_on="자치구")
final.head()

In [ ]:
# 구 이름을 index로 부여하기
final.set_index("구분", inplace=True)
final.head()

## 데이터 분석

### 상관분석


In [ ]:
# CCTV수와 인구수, 고령자비율, 외국인비율 간의 상관관계
# ?pd.DataFrame.corr
idx = ["총계", "인구수", "고령자비율", "외국인비율"]
final[idx].corr(method="spearman")

In [ ]:
# CCTV수와 인구수, 고령자비율, 외국인비율 간의 상관관계
# ?pd.DataFrame.corr
idx = ["총계", "인구수", "고령자비율", "외국인비율"]
final[idx].corr()

In [ ]:
?pd.DataFrame.corrwith

In [ ]:
final[idx].corrwith(final["총계"])

In [ ]:
# heatmap으로 나타내기
# heatmap이란? 범주형 자료의 계급(class)별로 연속형 자료를 집계한 자료를 사용하여,
# 집계한 값에 비례하여 색깔을 다르게 해서 2차원으로 자료를 시각화하는 그림이다.

?sns.heatmap

In [ ]:
sns.heatmap(final[idx].corr(), annot=True, fmt=".3f", linewidth=.5, cmap="Blues");

### 구별 CCTV 현황 그래프

In [ ]:
# 구별 CCTV 총계를 가로막대그래프로 그려보기
final.head()

In [ ]:
sns.barplot(x="총계", y=final.index, data=final);

In [ ]:
# 총계가 가장 높은 구부터 차례로 도표
final.sort_values("총계", ascending=False, inplace=True)
final.head()

In [ ]:
sns.barplot(x="총계", y=final.index, data=final);

In [ ]:
# 인구수 대비 CCTV 개수의 비율을 구하고 가장 높은 구부터 도표
final["CCTV비율"] = final["총계"] / final["인구수"] * 100
final.sort_values("CCTV비율", ascending=False, inplace=True)
sns.barplot(x="CCTV비율", y=final.index, data=final);

### 인구수와 CCTV 관계

In [ ]:
# 인구수 대비 CCTV 수에 대한 산점도 그리기
sns.relplot(kind="scatter", x="인구수", y="총계", data=final)
for i in range(final.shape[0]):
    plt.text(x=final["인구수"][i]*1.02, y=final["총계"][i]*0.98, s=final.index[i]); # s - The text (str)

In [ ]:
# 인구수 대비 CCTV 수에 대한 산점도 그리기 (CCTV가 가장 많은 구부터 5개구만 표기하기)
final.sort_values("총계", ascending=False, inplace=True)
sns.relplot(kind="scatter", x="인구수", y="총계", data=final)
for i in range(5):
    plt.text(x=final["인구수"][i]*1.02, y=final["총계"][i]*0.98, s=final.index[i]);

In [ ]:
# 인구수 대비 CCTV 개수에 대한 회귀도표 그리고 90% 신뢰구간 표기하기
sns.regplot(x="인구수", y="총계", data=final, ci=90);

# 일자리와 부동산 관계 분석

* [지역별 고용노동통계 데이터](http://laborstat.moel.go.kr/hmp/index.do)
* [KOSIS 주민등록세대수](https://kosis.kr/)
* [KOSIS 아파트 실거래 가격 지수](https://kosis.kr/)

* source: [파이썬으로 데이터 주무르기](https://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&barcode=9791158391546)


## 데이터 읽기


In [ ]:
# 지역별 일자리 데이터 읽어오기
job = pd.read_excel(dpath + "행정구역_시도__산업별_대분류__고용_2022.xlsx",  skiprows=1)
job.head()

In [ ]:
# 지역명 나누기
job["지역"] = job["지역별"].str[:4]
job["지역"]

In [ ]:
job["지역"].value_counts()

In [ ]:
job

In [ ]:
# 2022년 1월 데이터만 추출

job.rename(columns={job.columns[4]:"종사자수"}, inplace=True)
job1 = pd.DataFrame(job.loc[:,['종사자수','지역']])
job1

In [ ]:
# 세대수 데이터
pop = pd.read_excel(dpath + "행정구역_시군구_별_주민등록세대수_20221107013901.xlsx")
pop.head()

In [ ]:
# 지역별 데이터만 추출하기
pop["행정구역(시군구)별"]= pop["행정구역(시군구)별"].str[:4]
pop["행정구역(시군구)별"]

In [ ]:
pop1 = pd.DataFrame(pop[pop["행정구역(시군구)별"].isin(job["지역"])])
pop1

In [ ]:
# 2022년 1월 추출
pop1.rename(columns={pop1.columns[0]:"지역", pop1.columns[1]:"세대수"}, inplace=True)
pop1 = pd.DataFrame.reset_index(pop1.loc[:,['지역','세대수']])
pop1

In [ ]:
## 아파트 매매가격지수
apart = pd.read_excel(dpath + "아파트매매가격지수_시도_시_군_구__20221107013956.xlsx")
apart.head()

In [ ]:
apart["지역"] = apart["행정구역별"].str[:4]
apart.rename(columns={"2022.01":"아파트매매가지수"}, inplace=True)
apart1 = pd.DataFrame.reset_index(apart.loc[:,['지역','아파트매매가지수']])
apart1



## 데이터 병합

In [ ]:
temp = pd.merge(job1, pop1, on="지역")
df = pd.merge(temp, apart1, on="지역")
df.head()

In [ ]:
df = df[["지역", "세대수", "종사자수", "아파트매매가지수"]]
df.head()

## 데이터 분석


In [ ]:
# 시도별 고용자수를 막대그래프로 그리기
sns.barplot(x="지역", y="종사자수", data=df);
plt.xticks(rotation='vertical');

In [ ]:
# 고용자수 높은 시도부터 도표
df.sort_values("종사자수", ascending=False, inplace=True)
sns.barplot(x="지역", y="종사자수", data=df)
plt.xticks(rotation='vertical');

In [ ]:
# 고용자수 높은 시도부터 도표 (평균 고용자수 라인 추가)
sns.barplot(x="지역", y="종사자수", data=df, color="steelblue")
plt.axhline(y=df["종사자수"].mean(), c="orange", linewidth=2, linestyle="--")
plt.xticks(rotation='vertical');

In [ ]:
# 세대수 대비 고용비율 컬럼 추가
df["세대수대비고용률"] = df["종사자수"] / df["세대수"] * 100
df.head()

In [ ]:
# 시도 단위 세대수 대비 고용비율 도표
df.sort_values("세대수대비고용률", ascending=False, inplace=True)
sns.barplot(x="지역", y="세대수대비고용률", data=df, color="seagreen")
plt.axhline(y=df["세대수대비고용률"].mean(), c="orange", lw=1.5, ls=":")
plt.xticks(rotation='vertical');

In [ ]:
# 세대수 대비 고용률과 아파트 매매가지수 회귀도표
sns.lmplot(x="세대수대비고용률", y="아파트매매가지수", data=df)
for i in range(df.shape[0]):
    plt.text(x=df["세대수대비고용률"][i]*1.005, y=df["아파트매매가지수"][i], s=df["지역"][i]);

In [ ]:
# 시도별 고용자수, 세대수를 아파트 매매가지수와 함께 살펴보기
sns.relplot(kind="scatter", data=df, x="종사자수", y="세대수", hue="아파트매매가지수", size="아파트매매가지수", sizes=(40, 500));
for i in range(df.shape[0]):
    plt.text(x=df["종사자수"][i]*1.06, y=df["세대수"][i]*0.94, s=df["지역"][i]);

In [ ]:
# 시도별 세대수 대비 고용률, 세대수를 아파트 매매가지수와 함께 살펴보기 ()
sns.relplot(kind="scatter", data=df, x="세대수대비고용률", y="세대수", hue="아파트매매가지수", size="아파트매매가지수", sizes=(40, 500))
for i in range(df.shape[0]):
    plt.text(x=df["세대수대비고용률"][i]*1.02, y=df["세대수"][i]*0.98, s=df["지역"][i]);

# THE END